# Analyzing INTERNET speeds in Morocco

In [4]:

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import re

from tqdm.notebook import tqdm  # progress bar in Jupyter
from datetime import datetime
from shapely.geometry import Point
from adjustText import adjust_text

---

## Download data

First, download OOKLA data

In [2]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")
    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)

def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")
    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)

def quarter_end(year: int, q: int) -> datetime:
    if q == 4:
        return datetime(year + 1, 1, 1)
    return quarter_start(year, q + 1)

def get_tile_url(service_type: str, year: int, q: int) -> str | None:
    dt = quarter_start(year, q)
    end_dt = quarter_end(year, q)
    now = datetime.utcnow()

    if now < end_dt:
        print(f"⏩ Skipping {service_type} {year} Q{q} (quarter not yet complete)")
        return None

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

def download_file_with_progress(url: str, output_dir: str = "data") -> str:
    os.makedirs(output_dir, exist_ok=True)
    
    local_filename = os.path.join(output_dir, url.split("/")[-1])
    
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    t = tqdm(total=total_size, unit='iB', unit_scale=True, desc=f"Downloading {os.path.basename(local_filename)}")

    with open(local_filename, 'wb') as f:
        for data in response.iter_content(block_size):
            t.update(len(data))
            f.write(data)

    t.close()

    if total_size != 0 and t.n != total_size:
        print("⚠️ WARNING: Download might be incomplete.")
    
    return local_filename

configure ctype,years,and quarters as you need

In [3]:
ctype = ["fixed","mobile"]
years = [2023,2025]
quarters = [1, 2, 3, 4]
for t in ctype :
    for year in years :
        for q in quarters :
            url = get_tile_url(t, year, q)
           # download_file_with_progress(url) #uncomment to download

⏩ Skipping fixed 2025 Q2 (quarter not yet complete)
⏩ Skipping fixed 2025 Q3 (quarter not yet complete)
⏩ Skipping fixed 2025 Q4 (quarter not yet complete)
⏩ Skipping mobile 2025 Q2 (quarter not yet complete)
⏩ Skipping mobile 2025 Q3 (quarter not yet complete)
⏩ Skipping mobile 2025 Q4 (quarter not yet complete)


/tmp/ipykernel_145205/3529908371.py:21: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


In [4]:
Morocco = gp.read_file("data/Morocco/morocco.shp")

In [5]:

# List all matching zip files
zip_files = [f for f in os.listdir("data") if f.endswith(".zip") and "tiles" in f]

# Progress bar for loading
for filename in tqdm(zip_files, desc="Loading tiles"):
    path = os.path.join("data", filename)
    try:
        gdf = gp.read_file(path)
        print(f"LOADED : {filename}: {len(gdf)} features")
        name = filename
        gdf = gdf.to_crs(Morocco.crs)
        morocco_tiles = gp.clip(gdf, Morocco)
        output_name = name.replace(".zip", "_morocco.shp")
        output_path = os.path.join("MoroccoData", output_name)
        morocco_tiles.to_file(output_path, driver="ESRI Shapefile")
        print(f"SAVED: {output_path} ({len(morocco_tiles)} features)")
    except Exception as e:
        print(f"❌ Failed to load {filename}: {e}")
    

📥 Loading tiles:   0%|          | 0/10 [00:00<?, ?it/s]

✅ 2025-01-01_performance_fixed_tiles.zip: 6364159 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2025-01-01_performance_fixed_tiles_morocco.shp (18874 features)
✅ 2024-07-01_performance_mobile_tiles.zip: 3773658 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2024-07-01_performance_mobile_tiles_morocco.shp (12335 features)
✅ 2024-01-01_performance_mobile_tiles.zip: 3674000 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2024-01-01_performance_mobile_tiles_morocco.shp (12563 features)
✅ 2025-01-01_performance_mobile_tiles.zip: 3388115 features | CRS: EPSG:4326 | Columns: ['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices', 'geometry']
✅ Saved: MoroccoData/2025-01-01_performance_mobi

extracted from link[ https://github.com/teamookla/ookla-open-data/edit/master/README.md ]

#### Tile Attributes
Each tile contains the following adjoining attributes:

| Field Name        | Type        | Description                                                                                                                             | Notes                                                                                                                                                                              |
|-------------------|-------------|-----------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `avg_d_kbps`      | Integer     | The average download speed of all tests performed in the tile, represented in kilobits per second.                                      |                                                                                                                                                                                    |
| `avg_u_kbps`      | Integer     | The average upload speed of all tests performed in the tile, represented in kilobits per second.                                        |                                                                                                                                                                                    |
| `avg_lat_ms`      | Integer     | The average latency of all tests performed in the tile, represented in milliseconds                                                     |                                                                                                                                                                                    |
| `avg_lat_down_ms` | Integer     | The average latency under load of all tests performed in the tile as measured during the download phase of the test. Represented in ms. | Parquet-only. Added 2023-02-23 beginning in Q4 2022 dataset. This column is sparsely populated-- some rows will have a null value as not all versions of Speedtest can perform this measurement. |
| `avg_lat_up_ms`   | Integer     | The average latency under load of all tests performed in the tile as measured during the upload phase of the test. Represented in ms.   | Parquet-only. Added 2023-02-23 beginning in Q4 2022 dataset. This column is sparsely populated-- some rows will have a null value as not all versions of Speedtest can perform this measurement. |
| `tests`           | Integer     | The number of tests taken in the tile. |
| `devices`         | Integer     | The number of unique devices contributing tests in the tile. |
| `quadkey`         | Text        | The quadkey representing the tile.  |
| `tile_x`			| Numeric	  | X coordinate of the tile's centroid.| Added 2023-07-01 beginning in the Q3 2023 dataset.
| `tile_y`          | Numeric     | Y coordinate of the tile's centroid.| Added 2023-07-01 beginning in the Q3 2023 dataset.


#### Quadkeys

[Quadkeys](https://docs.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system) can act as a unique identifier for the tile. This can be useful for joining data spatially from multiple periods (quarters), creating coarser spatial aggregations without using geospatial functions, spatial indexing, partitioning, and an alternative for storing and deriving the tile geometry.

#### Layers
Two layers are distributed as separate sets of files:

* `performance_mobile_tiles` - Tiles containing tests taken from mobile devices with GPS-quality location and a cellular connection type (e.g. 4G LTE, 5G NR).
* `performance_fixed_tiles` - Tiles containing tests taken from mobile devices with GPS-quality location and a non-cellular connection type (e.g. WiFi, ethernet).

#### Time Period and Update Frequency

Layers are generated based on a quarter year of data (three months) and files will be updated and added on a quarterly basis. A `/year=2020/quarter=1/` period, the first quarter of the year 2020, would include all data generated on or after `2020-01-01` and before `2020-04-01`.

Data is subject to be reaggregated regularly in order to honor Data Subject Access Requests (DSAR) as is applicable in certain jurisdictions under laws including but not limited to General Data Protection Regulation (GDPR), California Consumer Privacy Act (CCPA), and Lei Geral de Proteção de Dados (LGPD). Therefore, data accessed at different times may result in variation in the total number of tests, tiles, and resulting performance metrics.



## LOAD MOROCCO DATA

In [2]:

DATA_FOLDER = "MoroccoData"

# Grab only shapefiles that end with "_morocco.shp"
Morocco_Data_files = [
    f for f in os.listdir(DATA_FOLDER) 
    if f.endswith("_morocco.shp")
]

# This will accumulate your “wide” data
final_gdf = None

# Regex to capture:
#   2024-01-01_performance_fixed_tiles_morocco.shp
#   group(1) => 2024-01-01
#   group(2) => fixed or mobile
filename_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})_.*?(fixed|mobile).*?_morocco\.shp')

for filename in tqdm(Morocco_Data_files, desc="Loading Morocco tiles"):
    path = os.path.join(DATA_FOLDER, filename)
    try:
        gdf = gp.read_file(path)
        print(f"LOADED : {filename}: {len(gdf)} features")

        # Extract quarter/date and type from filename
        match = filename_pattern.search(filename)
        if not match:
            print(f"❌ Could not parse quarter/type from {filename}")
            continue
        
        quarter_str = match.group(1)  # e.g. "2024-01-01"
        conn_type  = match.group(2)   # e.g. "fixed" or "mobile"
        
        # Build a rename map. We keep 'quadkey' and 'geometry' the same
        rename_map = {}
        for col in gdf.columns:
            if col not in ['quadkey', 'geometry']:
                # Prepend <quarter>_<type>_
                new_col = f"{quarter_str}_{conn_type}_{col}"
                rename_map[col] = new_col
        
        # Rename columns
        gdf_renamed = gdf.rename(columns=rename_map)
        
        # If final_gdf is None, just store this one.
        if final_gdf is None:
            final_gdf = gdf_renamed
        else:
            # We'll do a full outer merge only on 'quadkey'
            # Because tile_x & tile_y do not exist in your shapefiles
            gdf_no_geom = gdf_renamed.drop(columns=['geometry'])
            final_gdf = final_gdf.merge(
                gdf_no_geom,
                on='quadkey',
                how='outer'
            )
    except Exception as e:
        print(f"❌ Failed to load {filename}: {e}")

# At this point, final_gdf is your “wide” GeoDataFrame with columns like:
#   quadkey, geometry, 2024-01-01_fixed_avg_d_kbps, 2024-01-01_fixed_avg_u_kbps, etc.

# Save as a GeoPackage instead of ESRI Shapefile so you don’t lose column names:
final_out = "all_quarters_combined.gpkg"
final_gdf.to_file(final_out, driver="GPKG")
print(f"✅ Saved combined dataset to: {final_out}")


# -----------------------------------------
# Create a second file that has differences 
# between consecutive quarters for each type
import pandas as pd

diff_gdf = final_gdf.copy()

# We’ll parse all columns that match e.g. 2024-01-01_fixed_avg_d_kbps
col_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})_(fixed|mobile)_(.*)')

# Build a structure to group columns by (type, metric), e.g. (fixed, avg_d_kbps).
quarter_type_to_metrics = {}

for col in diff_gdf.columns:
    m = col_pattern.match(col)
    if m:
        q_str = m.group(1)
        t_str = m.group(2)
        metric_name = m.group(3)
        
        # Keep track of columns by quarter
        quarter_type_to_metrics.setdefault((t_str, metric_name), []).append(q_str)

# For each (type, metric), sort quarters & build difference columns
for (conn_type, metric_name), quarter_list in quarter_type_to_metrics.items():
    quarter_list_sorted = sorted(quarter_list)
    for i in range(1, len(quarter_list_sorted)):
        prev_q = quarter_list_sorted[i-1]
        curr_q = quarter_list_sorted[i]
        
        prev_col = f"{prev_q}_{conn_type}_{metric_name}"
        curr_col = f"{curr_q}_{conn_type}_{metric_name}"
        
        # If for some reason we’re missing a column, skip
        if prev_col not in diff_gdf.columns or curr_col not in diff_gdf.columns or metric_name in ['tests', 'devices']:
            continue
        
        diff_col_name = f"D_{curr_q}_M_{prev_q}_{conn_type}_{metric_name}"
        diff_gdf[diff_col_name] = diff_gdf[curr_col] - diff_gdf[prev_col]

# Save differences as well, again in a GeoPackage
diff_out = "all_quarters_differences.gpkg"
diff_gdf.to_file(diff_out, driver="GPKG")
print(f"✅ Saved differences dataset to: {diff_out}")

Loading Morocco tiles:   0%|          | 0/10 [00:00<?, ?it/s]

LOADED : 2024-04-01_performance_fixed_tiles_morocco.shp: 18484 features
LOADED : 2024-07-01_performance_fixed_tiles_morocco.shp: 18587 features
LOADED : 2024-10-01_performance_fixed_tiles_morocco.shp: 18414 features
LOADED : 2024-04-01_performance_mobile_tiles_morocco.shp: 13523 features
LOADED : 2024-01-01_performance_fixed_tiles_morocco.shp: 19127 features
LOADED : 2024-07-01_performance_mobile_tiles_morocco.shp: 12335 features
LOADED : 2025-01-01_performance_fixed_tiles_morocco.shp: 18874 features
LOADED : 2024-10-01_performance_mobile_tiles_morocco.shp: 12340 features
LOADED : 2025-01-01_performance_mobile_tiles_morocco.shp: 12002 features
LOADED : 2024-01-01_performance_mobile_tiles_morocco.shp: 12563 features
✅ Saved combined dataset to: all_quarters_combined.gpkg
✅ Saved differences dataset to: all_quarters_differences.gpkg


In [ ]:
import geopandas as gpd

# 1. Chemin vers votre shapefile
shp_path = "C:\Users\Ahmed\OoklaOpenDataMorocco\populaion_commune.shp"  

# 2. Chargement
communes = gpd.read_file(shp_path)

# 3. Exploration rapide
print(communes.head())
print("CRS :", communes.crs)
print("Colonnes disponibles :", communes.columns.tolist())

DataSourceError: population_commune.shp: No such file or directory